notes for showing to faith: 





a

Make sure to run the notebook in the correct python environment, with the correct pacakges installed!
I am currenlty using my python enviroment 'diamonds' which runs python 3.10.4
TODO: don't know what packages i have installed oops
TODO: make a recipe python environment

In [1]:
import numpy as np
import pandas as pd

In [3]:
# following code is necessary as some functions used in the funcE class produce future warnings but i can't fix it as it is not my code
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# importing the custom classes i built
from matrix_class import ProteinMatrix # ppi matrix 
from cluster_class import AllClusters # dictionary to hold all clusters (in form number of cluster : list of proteins in that cluster)
from degreelist_class import DegreeList # creates a list of all proteins in order of their degree

In [6]:
from func_e.FUNC_E import FUNC_E  # a method that can do funtional enrichment analysis!
import func_e.vocabs.all as vocabs